<a href="https://colab.research.google.com/github/xjsh1997/STAT6289/blob/main/Midterm_6289.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#import package
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [5]:
#read the file and prepare the data
url='https://raw.githubusercontent.com/xjsh1997/STAT6289/main/SBP%20daily%20average.csv'
df = pd.read_csv(url)
print(df.info())
#time series the date by the first column of date
df['date'] = pd.to_datetime(df['date'])
df.set_axis(df['date'], inplace=True)
#plot the time series data
import plotly.express as px
fig = px.line(df, x='date', y="SBP")
fig.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1070 non-null   object 
 1   SBP     1070 non-null   float64
dtypes: float64(1), object(1)
memory usage: 16.8+ KB
None


In [20]:
#set the train set and the validation set
SBP_data = df['SBP'].values
SBP_data = SBP_data.reshape((-1,1))
#we use the 2/3 length as the training set
#rest of dateset as the validation set
split_percent = 0.75
split = int(split_percent*len(SBP_data))

SBP_trainset = SBP_data[:split]
SBP_validset = SBP_data[split:]

date_trainset = df['date'][:split]
date_validset = df['date'][split:]

print(len(SBP_trainset))
print(len(SBP_validset))

802
268


In [40]:
#set the look back memory as 30 measurements
look_back = 20

train_generator = TimeseriesGenerator(SBP_trainset, SBP_trainset, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(SBP_validset, SBP_validset, length=look_back, batch_size=1)

#build the LSTM model
model1 = Sequential()
model1.add(
    LSTM(100,
        activation='relu',
        input_shape=(look_back,1))
)
#set loss fucntion as MSE
model1.add(Dense(1))
model1.compile(optimizer='adam', loss='mse')

num_epochs = 30
model1.fit_generator(train_generator, epochs=num_epochs, verbose=1)



Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



40/40 [==============================] - 2s 17ms/step - loss: 1649.6582
Epoch 2/30
40/40 [==============================] - 1s 18ms/step - loss: 136.1331
Epoch 3/30
40/40 [==============================] - 1s 17ms/step - loss: 19.5867
Epoch 4/30
40/40 [==============================] - 1s 18ms/step - loss: 15.6704
Epoch 5/30
40/40 [==============================] - 1s 17ms/step - loss: 14.9987
Epoch 6/30
40/40 [==============================] - 1s 17ms/step - loss: 17.9365
Epoch 7/30
40/40 [==============================] - 1s 17ms/step - loss: 14.7674
Epoch 8/30
40/40 [==============================] - 1s 17ms/step - loss: 13.8689
Epoch 9/30
40/40 [==============================] - 1s 17ms/step - loss: 18.1712
Epoch 10/30
40/40 [==============================] - 1s 18ms/step - loss: 14.0386
Epoch 11/30
40/40 [==============================] - 1s 17ms/step - loss: 13.2347
Epoch 12/30
40/40 [==============================] - 1s 17ms/step - loss: 13.4812
Epoch 13/30
40/40 [==============

In [46]:
#model2 by adding the layers
model2 = Sequential()

# Add the first layer.... the input shape is (Sample, seq_len-1, 1)
model2.add(LSTM(
        input_shape = (look_back, 1), return_sequences = True,
        units = 50))

# Add the second layer.... the input shape is (Sample, seq_len-1, 1)
model2.add(LSTM(
        input_shape = (look_back, 1), 
        units = 50))

# Add the output layer, simply one unit
model2.add(Dense(
        units = 1,
        activation = 'relu'))

model2.compile(loss = 'mse', optimizer = 'adam')
num_epochs = 20
model2.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



40/40 [==============================] - 5s 22ms/step - loss: 15960.1953
Epoch 2/20
40/40 [==============================] - 1s 23ms/step - loss: 14545.5508
Epoch 3/20
40/40 [==============================] - 1s 22ms/step - loss: 13789.2725
Epoch 4/20
40/40 [==============================] - 1s 23ms/step - loss: 13249.6035
Epoch 5/20
40/40 [==============================] - 1s 23ms/step - loss: 12756.4561
Epoch 6/20
40/40 [==============================] - 1s 28ms/step - loss: 12261.7617
Epoch 7/20
40/40 [==============================] - 1s 23ms/step - loss: 11795.3477
Epoch 8/20
40/40 [==============================] - 1s 23ms/step - loss: 11345.5938
Epoch 9/20
40/40 [==============================] - 1s 23ms/step - loss: 10923.1523
Epoch 10/20
40/40 [==============================] - 1s 23ms/step - loss: 10509.9219
Epoch 11/20
40/40 [==============================] - 1s 24ms/step - loss: 10107.3193
Epoch 12/20
40/40 [==============================] - 1s 24ms/step - loss: 9724.7061
E

In [48]:
import plotly.express as px
import plotly.graph_objs as go
#prediction visulization
#collect the prediction result from single layer and multi-layer model
SBP_prediction = model1.predict_generator(test_generator)
SBP_prediction2 = model2.predict_generator(test_generator)

SBP_trainset = SBP_trainset.reshape((-1))
SBP_validset = SBP_validset.reshape((-1))
SBP_prediction = SBP_prediction.reshape((-1))
SBP_prediction2 = SBP_prediction2.reshape((-1))


trace1 = go.Scatter(
    x = date_trainset,
    y = SBP_trainset,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_validset,
    y = SBP_prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_validset,
    y = SBP_validset,
    mode='lines',
    name = 'Ground Truth'
)
trace4 = go.Scatter(
    x = date_validset,
    y = SBP_prediction2,
    mode = 'lines',
    name = 'Prediction_2'
)
layout = go.Layout(
    title = "SBP forecating",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "SBP(mmHg)"}
)
fig = go.Figure(data=[trace1, trace2, trace3,trace4], layout=layout,layout_yaxis_range=[110,150])
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.

